In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 22 17:46:02 2022

@author: Lin
"""

import os
current_path = os.getcwd()+'/'
import sys
sys.path.append(current_path)
sys.path.append(current_path+'../../')
import numpy as np
import math

import PACKAGE_MP_Linear as Linear_2D
import PACKAGE_MP_Vertex as Vertex_2D
import PACKAGE_MP_3DLinear as Linear_3D
import PACKAGE_MP_3DVertex as Vertex_3D
import myInput

In [32]:

def test_vertex_3d():
    nx, ny, nz = 200, 200, 200
    ng = 2
    cores = 8
    max_iteration = 20
    radius = [5, 20, 50, 80, 2, 1]

    for r in radius:
        filename_save = f"./VT3D_Curvature_R{r}_Iteration_1_{max_iteration}"

        VT_errors =np.zeros(max_iteration)
        VT_runningTime = np.zeros(max_iteration)

        # P0,R=myInput.init2IC(nx, ny, ng, "PolyIC.init")
        P0,R=myInput.Circle_IC3d(nx,ny,nz,r)
        # P0,R=myInput.Voronoi_IC(nx,ny,ng)

        for cores in [cores]:
            # loop_times=10
            for interval in range(1,max_iteration):


                test1 = Vertex_3D.vertex3d_class(nx,ny,nz,ng,cores,interval,P0,R)
                # test1.linear_main()
                # P = test1.get_P()

                test1.vertex3d_main("curvature")
                C_ln = test1.get_C()

                # error

                print('loop_times = ' + str(test1.interval))
                print('running_time = %.2f' % test1.running_time)
                print('running_core time = %.2f' % test1.running_coreTime)
                print('total_errors = %.2f' % test1.errors)
                print('per_errors = %.3f' % test1.errors_per_site)
                print()


                VT_errors[interval-1] = test1.errors_per_site
                VT_runningTime[interval-1] = test1.running_coreTime

        np.savez(filename_save, VT_errors=VT_errors, VT_runningTime=VT_runningTime)



In [19]:


nx, ny, nz = 200,200,200
r = 10
P0,R = myInput.Circle_IC3d(nx,ny,nz,r)
P = np.zeros((4,nx,ny,nz))
for i in range(0,np.shape(P0)[3]):
  P[0,:,:,:] += P0[:,:,:,i]*(i+1)

voxel_num_edge_inner = 0
voxel_num_edge_outer = 0
face_num_inner = 0
face_num_outer = 0
for i in range(nx):
  for j in range(ny):
    for k in range(nz):
      ip,im,jp,jm,kp,km = myInput.periodic_bc3d(nx,ny,nz,i,j,k)
      nn_id = np.array([P[0,ip,j,k],P[0,im,j,k],P[0,i,jp,k],P[0,i,jm,k],P[0,i,j,kp],P[0,i,j,km]])
      nn_id_connect = (nn_id - P[0,i,j,k]) != 0
      nn_id_unique_num = len(np.unique(nn_id))
      if nn_id_unique_num > 2: continue # ignore the TJs and QJs
      if np.sum(nn_id_connect) > 0:

        if P[0,i,j,k] == 1:
          # figure out the num of faces for each sites
          num_faces = np.sum(nn_id_connect)
          face_num_inner += num_faces
          # figure out the number of edges
          if num_faces == 2: voxel_num_edge_outer += 1
          if num_faces == 3: voxel_num_edge_outer += 3
          if num_faces == 4: voxel_num_edge_outer += 5
          if num_faces == 5: voxel_num_edge_outer += 8
          if num_faces == 6: voxel_num_edge_outer += 12
        elif P[0,i,j,k] == 2:
          num_faces = np.sum(nn_id_connect)
          face_num_outer += num_faces
          # figure out the number of edges
          if num_faces == 2: voxel_num_edge_inner += 1
          if num_faces == 3: voxel_num_edge_inner += 3
          if num_faces == 4: voxel_num_edge_inner += 5
          if num_faces == 5: voxel_num_edge_inner += 8
          if num_faces == 6: voxel_num_edge_inner += 12



In [20]:

curvature = np.pi/4 * (voxel_num_edge_outer - voxel_num_edge_inner) / face_num_inner
print(f"The inner voxel num is {voxel_num_edge_inner}, the outer is {voxel_num_edge_outer}, the face num os {face_num_inner}")
print(f"The curvature is {curvature}, the error is {abs(curvature-1/r)}, the error fraction is {abs(curvature-1/r)/curvature}")

The inner voxel num is 888, the outer is 1116, the face num os 1830
The curvature is 0.09785288593148535, the error is 0.002147114068514658, the error fraction is 0.021942266169012372


In [58]:

nx, ny, nz = 200,200,30
wave = 10
P0,R=myInput.Complex2G_IC3d(nx,ny,nz,wave)
P = np.zeros((4,nx,ny,nz))
for i in range(0,np.shape(P0)[3]):
  P[0,:,:,:] += P0[:,:,:,i]*(i+1)

voxel_num_edge_inner = 0
voxel_num_edge_outer = 0
face_num_inner = 0
face_num_outer = 0
R_list=[]
for i in range(nx):
  for j in range(ny):
    for k in range(5,25):
      ip,im,jp,jm,kp,km = myInput.periodic_bc3d(nx,ny,nz,i,j,k)
      nn_id = np.array([P[0,ip,j,k],P[0,im,j,k],P[0,i,jp,k],P[0,i,jm,k],P[0,i,j,kp],P[0,i,j,km]])
      nn_id_connect = (nn_id - P[0,i,j,k]) != 0
      nn_id_unique_num = len(np.unique(nn_id))
      if nn_id_unique_num > 2: continue # ignore the TJs and QJs
      if np.sum(nn_id_connect) > 0:
        R_list.append(R[i,j,k,3])
        if P[0,i,j,k] == 1:
          # figure out the num of faces for each sites
          num_faces = np.sum(nn_id_connect)
          face_num_inner += num_faces
          # figure out the number of edges
          if num_faces == 2: voxel_num_edge_outer += 1
          if num_faces == 3: voxel_num_edge_outer += 3
          if num_faces == 4: voxel_num_edge_outer += 5
          if num_faces == 5: voxel_num_edge_outer += 8
          if num_faces == 6: voxel_num_edge_outer += 12
        elif P[0,i,j,k] == 2:
          num_faces = np.sum(nn_id_connect)
          face_num_outer += num_faces
          # figure out the number of edges
          if num_faces == 2: voxel_num_edge_inner += 1
          if num_faces == 3: voxel_num_edge_inner += 3
          if num_faces == 4: voxel_num_edge_inner += 5
          if num_faces == 5: voxel_num_edge_inner += 8
          if num_faces == 6: voxel_num_edge_inner += 12




In [59]:

curvature = np.pi/4 * (voxel_num_edge_outer - voxel_num_edge_inner) / face_num_inner
print(f"The inner voxel num is {voxel_num_edge_inner}, the outer is {voxel_num_edge_outer}, the face num os {face_num_inner}")
error = 0
for i in range(len(R_list)):
  if R_list[i] == 0: continue
  error += abs(R_list[i] - curvature)/R_list[i]
error = error/len(R_list)
print(f"The curvature is {curvature}, the error fraction is {error}")

The inner voxel num is 90990, the outer is 90990, the face num os 119600
The curvature is 0.0, the error fraction is 0.015841956726246473


## A simple test for 2D curvature algorithm

In [17]:


nx, ny = 200,200
r = 10
P0,R = myInput.Circle_IC(nx,ny,r)
P = np.zeros((3,nx,ny))
for i in range(0,np.shape(P0)[2]):
  P[0,:,:] += P0[:,:,i]*(i+1)

voxel_num_edge_inner = 0
voxel_num_edge_outer = 0
face_num_inner = 0
face_num_outer = 0
for i in range(nx):
  for j in range(ny):
    ip,im,jp,jm = myInput.periodic_bc(nx,ny,i,j)
    nn_id = np.array([P[0,ip,j],P[0,im,j],P[0,i,jp],P[0,i,jm]])
    nn_id_connect = (nn_id - P[0,i,j]) != 0
    nn_id_unique_num = len(np.unique(nn_id))
    if nn_id_unique_num > 2: continue # ignore the TJs and QJs
    if np.sum(nn_id_connect) > 0:

      if P[0,i,j] == 1:
        # figure out the num of faces for each sites
        num_faces = np.sum(nn_id_connect)
        face_num_inner += num_faces
        # figure out the number of edges
        if num_faces == 2: voxel_num_edge_outer += 1
        if num_faces == 3: voxel_num_edge_outer += 2
        if num_faces == 4: voxel_num_edge_outer += 4
      elif P[0,i,j] == 2:
        num_faces = np.sum(nn_id_connect)
        face_num_outer += num_faces
        # figure out the number of edges
        if num_faces == 2: voxel_num_edge_inner += 1
        if num_faces == 3: voxel_num_edge_inner += 2
        if num_faces == 4: voxel_num_edge_inner += 4



In [18]:

curvature = 5*np.pi/8 * (voxel_num_edge_outer - voxel_num_edge_inner) / face_num_inner
print(f"The inner voxel num is {voxel_num_edge_inner}, the outer is {voxel_num_edge_outer}, the face num os {face_num_inner}")
print(f"The curvature is {curvature}, the error is {abs(curvature-1/r)}, the error fraction is {abs(curvature-1/r)/curvature}")

The inner voxel num is 16, the outer is 20, the face num os 76
The curvature is 0.1033418636049274, the error is 0.0033418636049274003, the error fraction is 0.03233794600127628
